# Ragas Evaluation with Llama Stack - Demo

This notebook demonstrates how to use the Ragas out-of-tree provider.


## 1. Setup and Imports


In [10]:
# Install dev packages if not already installed
# !pip install -e ".[dev]""

from datetime import datetime
from rich.pretty import pprint

from llama_stack_client import LlamaStackClient


## 2. Llama Stack Client Setup

- Make sure we have an inference model (model_type='llm')
- Make sure we have an embedding model (model_type='embedding')


In [11]:
client = LlamaStackClient(base_url="http://localhost:8321")

models = client.models.list()
pprint(models)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[
│   Model(
│   │   identifier='all-MiniLM-L6-v2',
│   │   metadata={'embedding_dimension': 384.0},
│   │   api_model_type='embedding',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='all-minilm:latest',
│   │   model_type='embedding'
│   ),
│   Model(
│   │   identifier='granite3.3:2b',
│   │   metadata={},
│   │   api_model_type='llm',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='granite3.3:2b',
│   │   model_type='llm'
│   ),
│   Model(
│   │   identifier='meta-llama/Llama-3.2-3B-Instruct',
│   │   metadata={},
│   │   api_model_type='llm',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='llama3.2:3b-instruct-fp16',
│   │   model_type='llm'
│   ),
│   Model(
│   │   identifier='phi3',
│   │   metadata={},
│   │   api_model_type='llm',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='phi3:latest',
│   │   model_type='llm'
│   )
]

## 4. Dataset Preparation

Create a sample RAG evaluation dataset. In a real scenario, you would load your own dataset.


In [12]:
# Sample Ragas evaluation dataset
evaluation_data = [
    {
        "user_input": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "retrieved_contexts": [
            "Paris is the capital and most populous city of France."
        ],
        "reference": "Paris",
    },
    {
        "user_input": "Who invented the telephone?",
        "response": "Alexander Graham Bell invented the telephone in 1876.",
        "retrieved_contexts": [
            "Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone."
        ],
        "reference": "Alexander Graham Bell",
    },
    {
        "user_input": "What is photosynthesis?",
        "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
        "retrieved_contexts": [
            "Photosynthesis is a process used by plants to convert light energy into chemical energy."
        ],
        "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
    },
]

## 5. Dataset Registration

Register the dataset with Llama Stack's Datasets API using the direct rows approach.


In [13]:
# Register the dataset
dataset_id = f"ragas_demo_dataset_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

dataset_response = client.datasets.register(
    dataset_id=dataset_id,
    purpose="eval/question-answer",  # RAG evaluation purpose
    source={"type": "rows", "rows": evaluation_data},
    metadata={
        "provider_id": "localfs",  # seems there's a bug in datasets
        "description": "Sample RAG evaluation dataset for Ragas demo",
        "size": len(evaluation_data),
        "format": "ragas",
        "created_at": datetime.now().isoformat(),
    },
)
pprint(dataset_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/datasets "HTTP/1.1 200 OK"


DatasetRegisterResponse(
│   identifier='ragas_demo_dataset_20250727_062310',
│   metadata={
│   │   'provider_id': 'localfs',
│   │   'description': 'Sample RAG evaluation dataset for Ragas demo',
│   │   'size': 3.0,
│   │   'format': 'ragas',
│   │   'created_at': '2025-07-27T06:23:10.341283'
│   },
│   provider_id='localfs',
│   purpose='eval/question-answer',
│   source=SourceRowsDataSource(
│   │   rows=[
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   type='rows'
│   ),
│   type='dataset',
│   provider_resource_id='ragas_demo_dataset_20250727_062310',
│   owner={'principal': '', 'attributes': {}}
)

## 6. Benchmark Registration

Register a benchmark that defines what metrics to use for evaluation.


In [14]:
benchmark_id = f"ragas_demo_benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

ragas_metrics = [
    "answer_relevancy",  # How relevant is the answer to the question?
    # "context_precision",     # How precise are the retrieved contexts?
    # "faithfulness",          # How faithful is the answer to the contexts?
    # "context_recall",        # How much of the ground truth is covered by contexts?
    # "answer_correctness"  # How correct is the answer compared to ground truth?
]

benchmark_response = client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=dataset_id,
    scoring_functions=ragas_metrics,
    # metadata={
    #     "provider": "ragas",
    #     "version": "1.0",
    #     "metrics_count": len(ragas_metrics),
    #     "created_at": datetime.now().isoformat()
    # }
)

pprint(benchmark_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/eval/benchmarks "HTTP/1.1 200 OK"


None

In [15]:
benchmarks = client.benchmarks.list()
pprint(benchmarks[-2:])

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks "HTTP/1.1 200 OK"


[
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset_20250727_062215',
│   │   identifier='ragas_demo_benchmark_20250727_062215',
│   │   metadata={},
│   │   provider_id='trustyai_ragas',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark_20250727_062215'
│   ),
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset_20250727_062310',
│   │   identifier='ragas_demo_benchmark_20250727_062310',
│   │   metadata={},
│   │   provider_id='trustyai_ragas',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark_20250727_062310'
│   )
]

## 7. Evaluation Execution

Run the evaluation using our Ragas out-of-tree provider.


In [16]:
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file (all-MiniLM-L6-v2)

job = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            # "model": "meta-llama/Llama-3.2-3B-Instruct",
            "model": "granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
    },
)
pprint(job)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250727_062310/jobs "HTTP/1.1 200 OK"


Job(job_id='2', status='in_progress', result=None)

## 8. Results Display


In [17]:
job = client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(job)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250727_062310/jobs/2 "HTTP/1.1 200 OK"


Job(job_id='2', status='in_progress', result=None)

In [19]:
# wait a bit for the job to complete
job = client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(job)
results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(results)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250727_062310/jobs/2 "HTTP/1.1 200 OK"


Job(
│   job_id='2',
│   status='completed',
│   result={
│   │   'generations': [
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   'scores': {
│   │   │   'answer_relevancy': {
│   │   │   │   'score_rows': [
│   │   │   │   │   {'score': 0.9567411190551698},
│   │   │   │   │   {'score': 0.9262222139890982},
│   │   │   │   │   {'score': 0.8774825604564382}
│   │   │   │   ],
│   │   │   │   'aggregated_results': {'answer_relevancy': 0.920148631166902}
│   │   │   }
│   │   }
│   }
)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250727_062310/jobs/2/result "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'user_input': 'What is the capital of France?',
│   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   'response': 'The capital of France is Paris.',
│   │   │   'reference': 'Paris'
│   │   },
│   │   {
│   │   │   'user_input': 'Who invented the telephone?',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   ],
│   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   'reference': 'Alexander Graham Bell'
│   │   },
│   │   {
│   │   │   'user_input': 'What is photosynthesis?',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   ],
│   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   }
│   ],
│   scores={
│   │   'answer_relevancy': ScoringResult(
│   │   │   aggregated_results={'answer_relevancy': 0.920148631166902},
│   │   │   score_rows=[
│   │   │   │   {'score': 0.9567411190551698},
│   │   │   │   {'score': 0.9262222139890982},
│   │   │   │   {'score': 0.8774825604564382}
│   │   │   ]
│   │   )
│   }
)